### Required Libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
# check unqiue latest feature id
model_data['latest_feature_id'].nunique()

478328

### Drop Null target records

In [5]:
model_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [6]:
# Drop Null values of Target 
df = model_data.dropna(subset=['60_in_15mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

### Split the Time interval

In [7]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [8]:
model_df1.shape, oot_df1.shape

((101984, 2831), (17343, 2831))

In [9]:
features_list = [
#bureau intutive
'num_PERSONAL LOAN_enq_last_24M',
'num_CONSUMER LOAN_enq_last_9M',
'num_of_loans_le_1lac_non_credit',
'num_CREDIT CARD_enq_last_3M',
'Total_outstanding_Balance_CREDIT CARD_opened_last_6M',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
 'Utilization_CREDIT CARD_opened_last_36M',
'highest_sanction_amount_non_credit',
'Total_high_credit',
'num_PL_LE_1_Lac_opened_in_last_3M',

# banking intutive

'cash_withdrawal_sum',
'balance_5th_day_M2',
'bounced_transactions_count',
'days_balance_lt_5000_M2',
'min_balance_last_1_months',
'num_credit_gt_salary',
'upi_trans_count',
'Max_Credit_Avg_M1_M2_M3',
'ratio_total_debit_to_total_credit_M3',
'avg_daily_balance_last_1_months',
'ratio_balance_after_2_days_max_credit_M3',
'diff_btw_stated_salary_vs_actual_salary',
'ratio_6th_day_balance_to_max_credit_M2',
'ratio_6th_day_balance_to_max_debit_M1',

                 
# Reconcilated Features
'percentage_ever_delinquent', 
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',
'month_year',
'60_in_15mob']

In [10]:
len(features_list)

32

In [11]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [12]:
oot_df = oot_df1[features_list]

In [13]:
model_df.shape, oot_df.shape

((101984, 32), (17343, 32))

In [14]:
model_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    90856
1.0    11128
Name: count, dtype: int64

In [15]:
oot_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    15505
1.0     1838
Name: count, dtype: int64

In [16]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [17]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [18]:
#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3029380/2233446063.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [19]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3029380/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [20]:
X_oot.shape,y_oot.shape

((17343, 30), (17343,))

### Split Model dataset into Train and Test

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [22]:
X_train.shape, X_test.shape

((61190, 30), (40794, 30))

In [23]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(48952, 30) (12238, 30) (40794, 30)


In [24]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)

In [25]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.4,
    'colsample_bytree': 0.4,
    'lambda': 5.3,
    'alpha': 4.5,
    'gamma': 4.8,
    'scale_pos_weight': 3.8,
    'random_state': 42
    
}



evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 700,
    early_stopping_rounds= 30,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.59924	eval-auc:0.60288
[10]	train-auc:0.65135	eval-auc:0.64944
[20]	train-auc:0.65049	eval-auc:0.64818
[30]	train-auc:0.64939	eval-auc:0.64596
[40]	train-auc:0.65162	eval-auc:0.64933
[50]	train-auc:0.65193	eval-auc:0.64948
[60]	train-auc:0.65270	eval-auc:0.64910
[70]	train-auc:0.65400	eval-auc:0.64979
[80]	train-auc:0.65490	eval-auc:0.65055
[90]	train-auc:0.65668	eval-auc:0.65195
[100]	train-auc:0.65781	eval-auc:0.65265
[110]	train-auc:0.65885	eval-auc:0.65297
[120]	train-auc:0.65964	eval-auc:0.65356
[130]	train-auc:0.66072	eval-auc:0.65392
[140]	train-auc:0.66148	eval-auc:0.65431
[150]	train-auc:0.66197	eval-auc:0.65442
[160]	train-auc:0.66295	eval-auc:0.65531
[170]	train-auc:0.66337	eval-auc:0.65549
[180]	train-auc:0.66406	eval-auc:0.65609
[190]	train-auc:0.66462	eval-auc:0.65663
[200]	train-auc:0.66516	eval-auc:0.65706
[210]	train-auc:0.66594	eval-auc:0.65758
[220]	train-auc:0.66650	eval-auc:0.65793
[230]	train-auc:0.66709	eval-auc:0.65835
[240]	train-auc:0.66765	eva

In [26]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.69, Gini: 0.38, KS: 0.28
Eval AUC: 0.67, Gini: 0.35, KS: 0.25
Test AUC: 0.67, Gini: 0.34, KS: 0.24
OOT AUC: 0.68, Gini: 0.37, KS: 0.27


In [27]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.4f}, KS: {train_ks:.4f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.4f}, KS: {test_ks:.4f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.4f}, KS: {oot_ks:.4f}")

Train AUC: 0.69, Gini: 0.3779, KS: 0.2762
Test AUC: 0.67, Gini: 0.3368, KS: 0.2412
OOT AUC: 0.68, Gini: 0.3664, KS: 0.2666


In [28]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
28                          Num_Enq_non_BANK_last_12M       230.0
10                  num_PL_LE_1_Lac_opened_in_last_3M       154.0
27                           No of enq in past 90days       145.0
5                highest_sanction_amount_credit_cards       145.0
26                       percentage_utilization_gt_75       113.0
1                       num_CONSUMER LOAN_enq_last_9M       110.0
11                                cash_withdrawal_sum       101.0
6                          Age_of_oldest_open_account        95.0
7             Utilization_CREDIT CARD_opened_last_36M        90.0
25                         percentage_ever_delinquent        84.0
13                         bounced_transactions_count        79.0
15                          min_balance_last_1_months        73.0
4   Total_outstanding_Balance_CREDIT CARD_opened_l...        69.0
8                  highest_sanction_amount_non_credit        59.0
29        

In [29]:
feature_importance_df

Feature  Importance
28                          Num_Enq_non_BANK_last_12M       230.0
10                  num_PL_LE_1_Lac_opened_in_last_3M       154.0
27                           No of enq in past 90days       145.0
5                highest_sanction_amount_credit_cards       145.0
26                       percentage_utilization_gt_75       113.0
1                       num_CONSUMER LOAN_enq_last_9M       110.0
11                                cash_withdrawal_sum       101.0
6                          Age_of_oldest_open_account        95.0
7             Utilization_CREDIT CARD_opened_last_36M        90.0
25                         percentage_ever_delinquent        84.0
13                         bounced_transactions_count        79.0
15                          min_balance_last_1_months        73.0
4   Total_outstanding_Balance_CREDIT CARD_opened_l...        69.0
8                  highest_sanction_amount_non_credit        59.0
29                     Num_of_installment_loan_opened        56.0
0                      num_PERSONAL LOAN_enq_last_24M        54.0
3                         num_CREDIT CARD_enq_last_3M        53.0
9                                   Total_high_credit        49.0
12                                 balance_5th_day_M2        48.0
17                                    upi_trans_count        41.0
21           ratio_balance_after_2_days_max_credit_M3        39.0
16                               num_credit_gt_salary        35.0
19               ratio_total_debit_to_total_credit_M3        26.0
24              ratio_6th_day_balance_to_max_debit_M1        26.0
20                    avg_daily_balance_last_1_months        26.0
23             ratio_6th_day_balance_to_max_credit_M2        23.0
18                            Max_Credit_Avg_M1_M2_M3        21.0
2                     num_of_loans_le_1lac_non_credit        18.0
14                            days_balance_lt_5000_M2        18.0
22            diff_btw_stated_salary_vs_actual_salary        18.0

In [30]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.271650         0.535257        0.799606        0.449567   4896   
9          0.170787         0.410144        0.449544        0.378275   4895   
8          0.139122         0.355309        0.378273        0.334472   4895   
7          0.114402         0.316982        0.334465        0.300791   4895   
6          0.093361         0.286266        0.300788        0.272754   4895   
5          0.083146         0.260212        0.272749        0.248182   4895   
4          0.069254         0.237540        0.248176        0.227729   4895   
3          0.062104         0.217379        0.227723        0.207886   4895   
2          0.043105         0.196123        0.207882        0.181722   4895   
1          0.032067         0.162132        0.181719        0.107068   4896   

        label_good  label_bad  
Decile                         
10            3566       1330  
9             4059        836  
8             4214        681  
7             4335        560  
6             4438        457  
5             4488        407  
4             4556        339  
3             4591        304  
2             4684        211  
1             4739        157

In [31]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.263971         0.532848        0.798524        0.448635   4080   
9          0.161069         0.409508        0.448591        0.377889   4079   
8          0.137534         0.354651        0.377880        0.333949   4079   
7          0.115196         0.316806        0.333941        0.300462   4080   
6          0.095857         0.286445        0.300457        0.273343   4079   
5          0.093160         0.260712        0.273334        0.248379   4079   
4          0.080392         0.237714        0.248369        0.227816   4080   
3          0.068154         0.217458        0.227816        0.207791   4079   
2          0.054915         0.196131        0.207789        0.182402   4079   
1          0.036029         0.162900        0.182396        0.109257   4080   

        label_good  label_bad  
Decile                         
10            3003       1077  
9             3422        657  
8             3518        561  
7             3610        470  
6             3688        391  
5             3699        380  
4             3752        328  
3             3801        278  
2             3855        224  
1             3933        147

In [32]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.265130         0.546314        0.787165        0.457210   1735   
9          0.151672         0.414243        0.457172        0.380852   1734   
8          0.148789         0.356291        0.380815        0.334441   1734   
7          0.108420         0.317011        0.334392        0.301038   1734   
6          0.092849         0.286557        0.301032        0.272794   1734   
5          0.087032         0.259944        0.272793        0.247492   1735   
4          0.069204         0.236948        0.247485        0.227368   1734   
3          0.055940         0.217578        0.227343        0.208873   1734   
2          0.043829         0.197365        0.208862        0.183045   1734   
1          0.036888         0.163335        0.182988        0.114771   1735   

        label_good  label_bad  
Decile                         
10            1275        460  
9             1471        263  
8             1476        258  
7             1546        188  
6             1573        161  
5             1584        151  
4             1614        120  
3             1637         97  
2             1658         76  
1             1671         64

In [33]:
#Save the Model
xgb_model.save_model('LeoPayu_BB_60_in_15mob.json')
print('Model Saved')

Model Saved


In [34]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BB_Train_Pred.csv',index=False)
Eval.to_csv('LeoPayu_BB_Eval_Pred.csv',index=False)
Test.to_csv('LeoPayu_BB_Test_Pred.csv',index=False)
Oot.to_csv('LeoPayu_BB_Oot_Pred.csv',index=False)
